In [1]:
import pandas as pd
import numpy as np

In [2]:
app = pd.read_csv("Data/Raw/application_train.csv")

bureau = pd.read_csv("Data/Raw/bureau.csv")
bureau_bal = pd.read_csv("Data/Raw/bureau_balance.csv")

prev = pd.read_csv("Data/Raw/previous_application.csv")
inst = pd.read_csv("Data/Raw/installments_payments.csv")
pos = pd.read_csv("Data/Raw/POS_CASH_balance.csv")
cc = pd.read_csv("Data/Raw/credit_card_balance.csv")


In [3]:
bb_agg = (
    bureau_bal.groupby("SK_ID_BUREAU")
    .agg(
        BB_MONTH_COUNT=("MONTHS_BALANCE", "count"),
        BB_MONTH_MIN=("MONTHS_BALANCE", "min"),
        BB_MONTH_MAX=("MONTHS_BALANCE", "max"),
        BB_STATUS_NUNIQUE=("STATUS", "nunique"),
    )
    .reset_index()
)


In [4]:
bureau_full = bureau.merge(bb_agg, on="SK_ID_BUREAU", how="left")


In [5]:
bureau_client = (
    bureau_full.groupby("SK_ID_CURR")
    .agg(
        BURO_CREDIT_COUNT=("SK_ID_BUREAU", "count"),
        BURO_AMT_CREDIT_SUM_MEAN=("AMT_CREDIT_SUM", "mean"),
        BURO_AMT_CREDIT_SUM_DEBT_MEAN=("AMT_CREDIT_SUM_DEBT", "mean"),
        BURO_BB_MONTH_COUNT_MEAN=("BB_MONTH_COUNT", "mean"),
    )
    .reset_index()
)


In [6]:
prev_client = (
    prev.groupby("SK_ID_CURR")
    .agg(
        PREV_APP_COUNT=("SK_ID_PREV", "count"),
        PREV_AMT_CREDIT_MEAN=("AMT_CREDIT", "mean"),
        PREV_CNT_PAYMENT_MEAN=("CNT_PAYMENT", "mean"),
        PREV_DAYS_DECISION_MEAN=("DAYS_DECISION", "mean"),
    )
    .reset_index()
)


In [7]:
inst2 = inst.copy()

inst2["PAYMENT_DIFF"] = inst2["AMT_PAYMENT"] - inst2["AMT_INSTALMENT"]
inst2["DAYS_LATE"] = inst2["DAYS_ENTRY_PAYMENT"] - inst2["DAYS_INSTALMENT"]
inst2["DPD"] = inst2["DAYS_LATE"].clip(lower=0)  # retard uniquement


In [8]:
inst_client = (
    inst2.groupby("SK_ID_CURR")
    .agg(
        INST_COUNT=("SK_ID_PREV", "size"),
        INST_DPD_MEAN=("DPD", "mean"),
        INST_DPD_MAX=("DPD", "max"),
        INST_PAYMENT_DIFF_MEAN=("PAYMENT_DIFF", "mean"),
    )
    .reset_index()
)


In [9]:
pos_client = (
    pos.groupby("SK_ID_CURR")
    .agg(
        POS_MONTH_COUNT=("MONTHS_BALANCE", "count"),
        POS_SK_DPD_MEAN=("SK_DPD", "mean"),
        POS_SK_DPD_MAX=("SK_DPD", "max"),
    )
    .reset_index()
)


In [10]:
cc_client = (
    cc.groupby("SK_ID_CURR")
    .agg(
        CC_MONTH_COUNT=("MONTHS_BALANCE", "count"),
        CC_AMT_BALANCE_MEAN=("AMT_BALANCE", "mean"),
        CC_AMT_CREDIT_LIMIT_MEAN=("AMT_CREDIT_LIMIT_ACTUAL", "mean"),
    )
    .reset_index()
)


In [11]:
df_final = (
    app
    .merge(bureau_client, on="SK_ID_CURR", how="left")
    .merge(prev_client, on="SK_ID_CURR", how="left")
    .merge(inst_client, on="SK_ID_CURR", how="left")
    .merge(pos_client, on="SK_ID_CURR", how="left")
    .merge(cc_client, on="SK_ID_CURR", how="left")
)

print(df_final.shape)
df_final.head()


(307511, 140)


,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,INST_COUNT,INST_DPD_MEAN,INST_DPD_MAX,INST_PAYMENT_DIFF_MEAN,POS_MONTH_COUNT,POS_SK_DPD_MEAN,POS_SK_DPD_MAX,CC_MONTH_COUNT,CC_AMT_BALANCE_MEAN,CC_AMT_CREDIT_LIMIT_MEAN
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,...,19.0,0.000000,0.0,0.000000,19.0,0.0,0.0,NaN,NaN,NaN
1,100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,...,25.0,0.000000,0.0,0.000000,28.0,0.0,0.0,NaN,NaN,NaN
2,100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,...,3.0,0.000000,0.0,0.000000,4.0,0.0,0.0,NaN,NaN,NaN
3,100006,0,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,...,16.0,0.000000,0.0,0.000000,21.0,0.0,0.0,6.0,0.0,270000.0
4,100007,0,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,...,66.0,0.954545,12.0,-452.384318,66.0,0.0,0.0,NaN,NaN,NaN


In [12]:
# 1 ligne = 1 client ?
print("Lignes df_final :", df_final.shape[0])
print("Clients uniques :", df_final['SK_ID_CURR'].nunique())

# Valeurs manquantes (top 10)
display(df_final.isnull().mean().sort_values(ascending=False).head(10))


Lignes df_final : 307511
Clients uniques : 307511


CC_MONTH_COUNT              0.717392
CC_AMT_BALANCE_MEAN         0.717392
CC_AMT_CREDIT_LIMIT_MEAN    0.717392
BURO_BB_MONTH_COUNT_MEAN    0.700073
COMMONAREA_AVG              0.698723
COMMONAREA_MEDI             0.698723
COMMONAREA_MODE             0.698723
NONLIVINGAPARTMENTS_AVG     0.694330
NONLIVINGAPARTMENTS_MODE    0.694330
NONLIVINGAPARTMENTS_MEDI    0.694330
dtype: float64

In [13]:
df_final.to_csv("Data/Processed/application_train_fused.csv", index=False)


In [14]:
#Feature engineering 

df_final["DAYS_EMPLOYED_PERC"] = df_final["DAYS_EMPLOYED"] / df_final["DAYS_BIRTH"]

df_final["INCOME_CREDIT_PERC"] = df_final["AMT_INCOME_TOTAL"] / df_final["AMT_CREDIT"]

df_final["INCOME_PER_PERSON"] = df_final["AMT_INCOME_TOTAL"] / df_final["CNT_FAM_MEMBERS"]

df_final["ANNUITY_INCOME_PERC"] = df_final["AMT_ANNUITY"] / df_final["AMT_INCOME_TOTAL"]

df_final["PAYMENT_RATE"] = df_final["AMT_ANNUITY"] / df_final["AMT_CREDIT"]


df_final.replace([np.inf, -np.inf], np.nan, inplace=True)


In [17]:
df_final.dtypes.value_counts()


float64    88
int64      41
object     16
Name: count, dtype: int64

In [19]:
df_final.select_dtypes(include="object").columns


Index(['NAME_CONTRACT_TYPE', 'CODE_GENDER', 'FLAG_OWN_CAR', 'FLAG_OWN_REALTY',
       'NAME_TYPE_SUITE', 'NAME_INCOME_TYPE', 'NAME_EDUCATION_TYPE',
       'NAME_FAMILY_STATUS', 'NAME_HOUSING_TYPE', 'OCCUPATION_TYPE',
       'WEEKDAY_APPR_PROCESS_START', 'ORGANIZATION_TYPE', 'FONDKAPREMONT_MODE',
       'HOUSETYPE_MODE', 'WALLSMATERIAL_MODE', 'EMERGENCYSTATE_MODE'],
      dtype='object')

In [23]:
cat_cols = df_final.select_dtypes(include="object").columns

df_final = pd.get_dummies(df_final, columns=cat_cols, dummy_na=True)


In [25]:
df_final.dtypes.value_counts()


bool       156
float64     88
int64       41
Name: count, dtype: int64